# Installation

In [1]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58f40000 @  0x7fde4e0742a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 23.9MB/s 
    100% |████████████████████████████████| 2.0MB 4.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [18]:
# File Uploader
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving helper.py to helper.py


# Inference and Validation

Now that you have a trained network, you can use it for making predictions. This is typically called **inference**, a term borrowed from statistics. However, neural networks have a tendency to perform *too well* on the training data and aren't able to generalize to data that hasn't been seen before. This is called **overfitting** and it impairs inference performance. To test for overfitting while training, we measure the performance on data not in the training set called the **validation** set. We avoid overfitting through regularization such as dropout while monitoring the validation performance during training. In this notebook, I'll show you how to do this in PyTorch. 

As usual, let's start by loading the dataset through torchvision. You'll learn more about torchvision and loading data in a later part. This time we'll be taking advantage of the test set which you can get by setting `train=False` here:

```python
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
```

The test set contains images just like the training set. Typically you'll see 10-20% of the original dataset held out for testing and validation with the rest being used for training.

In [2]:
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Processing...
Done!


Here I'll create a model like normal, using the same one from my solution for part 4.

In [0]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

The goal of validation is to measure the model's performance on data that isn't part of the training set. Performance here is up to the developer to define though. Typically this is just accuracy, the percentage of classes the network predicted correctly. Other options are [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall#Definition_(classification_context)) and top-5 error rate. We'll focus on accuracy here. First I'll do a forward pass with one batch from the test set.

In [33]:
model = Classifier()

images, labels = next(iter(testloader))
# Get the class probabilities
ps = torch.exp(model(images))
# Make sure the shape is appropriate, we should get 10 class probabilities for 64 examples
print(ps.shape)

torch.Size([64, 10])


With the probabilities, we can get the most likely class using the `ps.topk` method. This returns the $k$ highest values. Since we just want the most likely class, we can use `ps.topk(1)`. This returns a tuple of the top-$k$ values and the top-$k$ indices. If the highest value is the fifth element, we'll get back 4 as the index.

In [34]:
top_p, top_class = ps.topk(1, dim=1)
# Look at the most likely classes for the first 10 examples
print(top_class[:10,:])

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])


Now we can check if the predicted classes match the labels. This is simple to do by equating `top_class` and `labels`, but we have to be careful of the shapes. Here `top_class` is a 2D tensor with shape `(64, 1)` while `labels` is 1D with shape `(64)`. To get the equality to work out the way we want, `top_class` and `labels` must have the same shape.

If we do

```python
equals = top_class == labels
```

`equals` will have shape `(64, 64)`, try it yourself. What it's doing is comparing the one element in each row of `top_class` with each element in `labels` which returns 64 True/False boolean values for each row.

In [47]:
equals = top_class.view(top_class.shape[0]) == labels
print(top_class.shape)
print(labels.shape)
print(equals.shape)

torch.Size([64, 1])
torch.Size([64])
torch.Size([64])


Now we need to calculate the percentage of correct predictions. `equals` has binary values, either 0 or 1. This means that if we just sum up all the values and divide by the number of values, we get the percentage of correct predictions. This is the same operation as taking the mean, so we can get the accuracy with a call to `torch.mean`. If only it was that simple. If you try `torch.mean(equals)`, you'll get an error

```
RuntimeError: mean is not implemented for type torch.ByteTensor
```

This happens because `equals` has type `torch.ByteTensor` but `torch.mean` isn't implemented for tensors with that type. So we'll need to convert `equals` to a float tensor. Note that when we take `torch.mean` it returns a scalar tensor, to get the actual value as a float we'll need to do `accuracy.item()`.

In [48]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

Accuracy: 6.25%


The network is untrained so it's making random guesses and we should see an accuracy around 10%. Now let's train our network and include our validation pass so we can measure how well the network is performing on the test set. Since we're not updating our parameters in the validation pass, we can speed up our code by turning off gradients using `torch.no_grad()`:

```python
# turn off gradients
with torch.no_grad():
    # validation pass here
    for images, labels in testloader:
        ...
```

>**Exercise:** Implement the validation loop below and print out the total accuracy after the loop. You can largely copy and paste the code from above, but I suggest typing it in because writing it out yourself is essential for building the skill. In general you'll always learn more by typing it rather than copy-pasting. You should be able to get an accuracy above 80%.

In [5]:
# GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(device)

model = Classifier()
model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        ## TODO: Implement the validation pass and print out the validation accuracy
        
      with torch.no_grad():

        all_equality = []
        for images, labels in testloader:
          
          images, labels = images.to(device), labels.to(device)

          log_ps = model(images)
          probabilities = torch.exp(log_ps)

          top_p, preds = probabilities.topk(1, dim=1)
    
          test_loss = criterion(log_ps, labels)
          test_losses.append(test_loss)

          equality = torch.tensor(preds).view(preds.shape[0]) == labels 
          all_equality.extend(equality)

      accuracy = torch.mean(torch.tensor(all_equality).type(torch.FloatTensor))
      print(f'Epoch {e+1} - Accuracy: {accuracy.item()*100}%')

cuda:0
Epoch 1 - Accuracy: 84.52000021934509%
Epoch 2 - Accuracy: 84.24000144004822%
Epoch 3 - Accuracy: 85.31000018119812%
Epoch 4 - Accuracy: 85.39000153541565%
Epoch 5 - Accuracy: 86.51000261306763%
Epoch 6 - Accuracy: 86.4799976348877%
Epoch 7 - Accuracy: 86.90000176429749%
Epoch 8 - Accuracy: 87.48999834060669%
Epoch 9 - Accuracy: 87.16999888420105%
Epoch 10 - Accuracy: 87.41999864578247%
Epoch 11 - Accuracy: 87.43000030517578%
Epoch 12 - Accuracy: 87.74999976158142%
Epoch 13 - Accuracy: 88.09999823570251%
Epoch 14 - Accuracy: 87.8499984741211%
Epoch 15 - Accuracy: 87.40000128746033%
Epoch 16 - Accuracy: 87.95999884605408%
Epoch 17 - Accuracy: 87.90000081062317%
Epoch 18 - Accuracy: 88.52999806404114%
Epoch 19 - Accuracy: 87.47000098228455%
Epoch 20 - Accuracy: 86.79999709129333%
Epoch 21 - Accuracy: 88.2099986076355%
Epoch 22 - Accuracy: 88.48999738693237%
Epoch 23 - Accuracy: 88.53999972343445%
Epoch 24 - Accuracy: 87.72000074386597%
Epoch 25 - Accuracy: 88.71999979019165%
Epoch

## Overfitting

If we look at the training and validation losses as we train the network, we can see a phenomenon known as overfitting.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/overfitting.png?raw=1' width=450px>

The network learns the training set better and better, resulting in lower training losses. However, it starts having problems generalizing to data outside the training set leading to the validation loss increasing. The ultimate goal of any deep learning model is to make predictions on new data, so we should strive to get the lowest validation loss possible. One option is to use the version of the model with the lowest validation loss, here the one around 8-10 training epochs. This strategy is called *early-stopping*. In practice, you'd save the model frequently as you're training then later choose the model with the lowest validation loss.

The most common method to reduce overfitting (outside of early-stopping) is *dropout*, where we randomly drop input units. This forces the network to share information between weights, increasing it's ability to generalize to new data. Adding dropout in PyTorch is straightforward using the [`nn.Dropout`](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout) module.

```python
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        # output so no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x
```

During training we want to use dropout to prevent overfitting, but during inference we want to use the entire network. So, we need to turn off dropout during validation, testing, and whenever we're using the network to make predictions. To do this, you use `model.eval()`. This sets the model to evaluation mode where the dropout probability is 0. You can turn dropout back on by setting the model to train mode with `model.train()`. In general, the pattern for the validation loop will look like this, where you turn off gradients, set the model to evaluation mode, calculate the validation loss and metric, then set the model back to train mode.

```python
# turn off gradients
with torch.no_grad():
    
    # set model to evaluation mode
    model.eval()
    
    # validation pass here
    for images, labels in testloader:
        ...

# set model back to train mode
model.train()
```

> **Exercise:** Add dropout to your model and train it on Fashion-MNIST again. See if you can get a lower validation loss or higher accuracy.

In [0]:
## TODO: Define your model with dropout added
class MyClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 10)
    
    self.dropout = nn.Dropout(p=0.2)
    
  def forward(self, x):
    
    x = x.view(x.shape[0], -1)
    
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))
    
    x = F.log_softmax(self.fc4(x), dim=1)
    
    return x

In [16]:
## TODO: Train your model with dropout, and monitor the training progress with the validation loss and accuracy

model = MyClassifier()
model.to(device)

epochs = 30
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)


for epoch in range(epochs):
  
  train_losses, val_losses = [], []
  
  # Train
  model.train()
  for image, labels in trainloader:
    image, labels = image.to(device), labels.to(device)
    
    optimizer.zero_grad()
    
    logits = model(image)
    train_loss = criterion(logits, labels)
    train_loss.backward()
    optimizer.step()

    train_losses.append(train_loss)
    
    
  # Calculate val metrics after train epoch
  with torch.no_grad():
    model.eval()
    
    for images, labels in testloader:
      images, labels = images.to(device), labels.to(device)
      
      logits = model(images)
      val_loss = criterion(logits, labels)
      
      val_losses.append(val_loss)
      
  avg_train_loss = torch.mean(torch.tensor(train_losses)).item()
  avg_val_loss = torch.mean(torch.tensor(val_losses)).item()
  print(f"Epoch {epoch+1} - Train Loss: {round(avg_train_loss, 4)} / Val Loss: {round(avg_val_loss, 4)}")      
      

Epoch 1 - Train Loss: 0.6027 / Val Loss: 0.4658
Epoch 2 - Train Loss: 0.4841 / Val Loss: 0.4535
Epoch 3 - Train Loss: 0.4504 / Val Loss: 0.4519
Epoch 4 - Train Loss: 0.438 / Val Loss: 0.4114
Epoch 5 - Train Loss: 0.4225 / Val Loss: 0.3946
Epoch 6 - Train Loss: 0.4164 / Val Loss: 0.4079
Epoch 7 - Train Loss: 0.4029 / Val Loss: 0.4203
Epoch 8 - Train Loss: 0.3988 / Val Loss: 0.3848
Epoch 9 - Train Loss: 0.3925 / Val Loss: 0.3958
Epoch 10 - Train Loss: 0.3921 / Val Loss: 0.4196
Epoch 11 - Train Loss: 0.3821 / Val Loss: 0.393
Epoch 12 - Train Loss: 0.3808 / Val Loss: 0.4022
Epoch 13 - Train Loss: 0.3755 / Val Loss: 0.3983
Epoch 14 - Train Loss: 0.3828 / Val Loss: 0.3889
Epoch 15 - Train Loss: 0.371 / Val Loss: 0.3926
Epoch 16 - Train Loss: 0.3772 / Val Loss: 0.392
Epoch 17 - Train Loss: 0.3686 / Val Loss: 0.3738
Epoch 18 - Train Loss: 0.3677 / Val Loss: 0.382
Epoch 19 - Train Loss: 0.3575 / Val Loss: 0.3744
Epoch 20 - Train Loss: 0.358 / Val Loss: 0.3687
Epoch 21 - Train Loss: 0.3577 / Val

## Inference

Now that the model is trained, we can use it for inference. We've done this before, but now we need to remember to set the model in inference mode with `model.eval()`. You'll also want to turn off autograd with the `torch.no_grad()` context.

In [34]:
# Import helper module (should be in the repo)
import helper

# Test out your network!

model.to('cpu')
model.eval()

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]
# Convert 2D image to 1D vector
img = img.view(1, 784)

# Calculate the class probabilities (softmax) for img
with torch.no_grad():
    output = model.forward(img)

ps = torch.exp(output)

# Plot the image and probabilities
helper.view_classify(img.view(1, 28, 28), ps, version='Fashion')

## Next Up!

In the next part, I'll show you how to save your trained models. In general, you won't want to train a model everytime you need it. Instead, you'll train once, save it, then load the model when you want to train more or use if for inference.